In [25]:
!pip install -q sat-search stackstac stac-vrt rioxarray==0.3.1

## STAC + Dask

STAC and Dask pair nicely. Dask works best when it can build up a large chain of tasks and then execute that computation later on. When you're operating on the xarray DataArray the actual computation is lazy. But we *do* need some kind of "schema" to build up the original DataArray in the first place. Things like the sizes, coordinates, dimension names, etc.

You *can* do this with a bunch of COGs in blob storage, it's just slow. Whether your're stitching together a bunch of COGs are stacking them over time, you'll need to open each COG to read it's metadata (CRS, bands, bounding box, transform, etc.). Opening each COG from Blob Storage involves *at least* one HTTP request, often more.

The neat opportunity afforded by STAC is to get all of that metadata up front with just one or a handful of HTTP requests. We can construct the DataArray using just information gleaned from STAC, without opening a single COG. Then when it's time to do the actual computation, we start opening COGs, reading data, and operating on it.

So that's the opportunity: **efficiently create DataArrays from metadata in a STAC ItemCollection, rather than opening individual COGs**.

## Comparison between stackstac and stac-vrt

Both [`stackstac`](https://stackstac.readthedocs.io/) and [`stac-vrt`](https://stac-vrt.readthedocs.io/en/latest/) are exploiting this opportunity. This notebook explores some of their similarities and differences.

### stackstac example

`stackstac` exposes a single function, `stackstac.stack`, which -- you guessed it -- stacks a STAC ItemCollection. It's currently able to stack a *timeseries* of STAC items covering the roughly the same spatial extent.

In [2]:
# https://stackstac.readthedocs.io/en/latest/basic.html
import stackstac
import satsearch

lon, lat = -105.78, 35.79

items = satsearch.Search(
    url="https://earth-search.aws.element84.com/v0",
    intersects=dict(type="Point", coordinates=[lon, lat]),
    collections=["sentinel-s2-l2a-cogs"],
    datetime="2020-04-01/2020-05-01"
).items()
len(items)

13

There are 13 observations (time periods), each with 17 bands. They can be stacked into a 4-D cube with `stackstac.stack`.

In [3]:
ds = stackstac.stack(items)
ds

,Array,Chunk
Bytes,213.15 GB,8.39 MB
Shape,"(13, 17, 10980, 10980)","(1, 1, 1024, 1024)"
Count,27304 Tasks,26741 Chunks
Type,float64,numpy.ndarray


### stac-vrt example

`stac-vrt` also exposes a single function, `build_vrt` in this case. It takes a STAC ItemCollection and builds a GDAL VRT. It's currently focused on stitching together many images taken at the same time.

In [10]:
import requests

stac_items = requests.get(
    url="https://pct-pqe-staging.westeurope.cloudapp.azure.com/stac/v1/collections/usda-naip/items"
).json()["features"]
len(stac_items)

10

In [29]:
import stac_vrt

%time vrt = stac_vrt.build_vrt(stac_items, data_type="Byte")

CPU times: user 27.5 ms, sys: 0 ns, total: 27.5 ms
Wall time: 26.8 ms


`vrt` is a string that's a valid GDAL VRT.

In [32]:
print(vrt[:1000])

<VRTDataset rasterXSize="83790" rasterYSize="196870">
  <SRS dataAxisToSRSAxisMapping="1,2">PROJCS["NAD83 / UTM zone 17N",GEOGCS["NAD83",DATUM["North_American_Datum_1983",SPHEROID["GRS 1980",6378137,298.257222101,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","6269"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4269"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",-81],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","26917"]]</SRS>
  <GeoTransform>524382.0, 0.6, 0.0, 3097458.0, 0.0, -0.6</GeoTransform>
  <VRTRasterBand dataType="Byte" band="1">
    <ColorInterp>Red</ColorInterp>
    <SimpleSource>
      <SourceFilename relativeToVRT="0">/vsicurl/https://naipeuwest.blob.core.windows.net/naip/v002/fl/2019/fl_60cm_2

Libraries like rasterio and rioxarray know how to work with VRTs.

In [30]:
import rioxarray

ds = rioxarray.open_rasterio(vrt, chunks=(1, 4096, 4096))
ds

,Array,Chunk
Bytes,65.98 GB,16.78 MB
Shape,"(4, 196870, 83790)","(1, 4096, 4096)"
Count,4117 Tasks,4116 Chunks
Type,uint8,numpy.ndarray


## Comparision

1. **focus**: timeseries at a location vs. single timestep at many locations**

`stackstac` currently focuses on stacking a stack of STAC items along the `time` dimension. `stac-vrt` focuses on stitching together many slightly overlapping images from approximately the same time.

I suspect that `stackstac` can grow to handle the "stiching many images at a single timestep" case. I'm not sure if `stac-vrt` can grow to handle the "stack of images along time" case, mainly because I don't know if GDAL's VRT can (nested VRTs, especially without local files?).

2. **implementation**: build a `VRT` vs. build a `DataArray`

`stac-vrt` is a bit simpler. It's solely focused on building a VRT from an `ItemCollection`. It knows nothing about I/O, Dask, etc. It inherits all the limitations of VRTs.

`stackstac` is more complicated, but I think more flexible. It implements the code to do "this chunk of this Dask array comes from reading that chunk of that source file". I suspect that that's more flexible than `stack-vrt`.